In [46]:
###### IMPORTS ######
import pandas as pd
import nltk
import ssl

from nltk.tokenize import word_tokenize
from collections import defaultdict
import numpy as np
from sklearn.model_selection import train_test_split

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#nltk downloads
nltk.download('punkt')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/siddharthsakriya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [47]:
#now i want to import the data from /data folder
df = pd.read_csv('dataset/spam.csv')

In [48]:
#Now I am going to print out the first 5 rows of the data, and the shape.
print(df.head())
print(df.shape)
print(df['Category'].value_counts())


  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
(5572, 2)
Category
ham     4825
spam     747
Name: count, dtype: int64


In [49]:
###### CLEANING THE DATA ######

#drop any nan values if there is
df = df.dropna()

#now I need to loop through each row in the message column and use NLTK to tokenise the words
df['tokens'] = df['Message'].apply(nltk.word_tokenize)

#now I need to go through each row in the tokens column, remove any punctuation and make all the words lowercase, remove punctuation
df['tokens'] = df['tokens'].apply(lambda x: [word.lower() for word in x if word.isalpha()])

In [ ]:
class NaiveBayesClassifier:
    def __init__(self) -> None:
        #dictionary for prior probabilities
        self.priors = defaultdict(float)
        #dictionary for conditional probabilities
        self.conditionals = defaultdict(lambda: defaultdict(float))
        #vocab
        self.vocab = set()
        
    def train_model(self, x, y):
        #calculae the prior probabilities
        class_counts = defaultdict(float)
        for label in y:
            class_counts[label] += 1

        #use this to calculate the prior probabilities
        for label in class_counts:
            self.priors[label] = class_counts[label] / len(y)
        
        #calculate the conditional probs
        for label in class_counts:
            #get all rows with the same label
            label_rows = x[y == label]
            #flatten the list of lists
            all_words = [word for row in label_rows for word in row]
            word_counts = defaultdict(float)
            

         
        
        
        

        